# Load Packages

In [1]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_prediction_df_dict, save_pi_df_dict
from src.file_processing.processing_predictions import load_pi_df_dict
from src.file_manager.save_load_scaler import load_scaler
from src.pi_methods.knn import knn_prediction_interval
from src.pi_methods.weighted import weighted_prediction_interval
from src.pi_methods.cond_gaussian import cond_gauss_prediction_interval
from src.pi_methods.gaussian_copula import gauss_copula_prediction_interval
from src.pi_methods.conformal_prediction import conformal_prediction_interval
from src.evaluation.investigate_re_pe_distribution import investigate_re_n_pe_for_all_seeds, investigate_re_n_pe
from src.misc import create_folder
from ue_pi_dicts import ue_dict
# from seed_file import seed

seed_list = [2023, 2024, 2025, 2026, 2027]
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_processed_data_folder = join(fp_project_folder, "processed_data")
fp_output_data_folder = join(fp_processed_data_folder, "physionet")
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")

pred_file_names =  ["rue", "gpr", "infernoise", "der", "bnn"]

2025-06-05 14:29:17.144846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 14:29:17.156927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 14:29:17.160567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 14:29:17.170406: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-05 14:29:17.787977: W tensorflow/compiler/tf2

# Load Data

In [2]:
split_dict = load_split_dict(fp_output_data_folder)
scaler = load_scaler(join(fp_output_data_folder, "scaler.pickle"))

# Investigate Recon-Pred Error Space

In [6]:
investigate_re_n_pe_for_all_seeds(
    seed_list=seed_list, split_dict=split_dict, fp_pi_predictions=fp_pi_predictions)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Loaded df_dict!


  0%|          | 0/3 [00:00<?, ?it/s]

/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/sc

...

  0%|          | 0/3 [00:00<?, ?it/s]

Loaded df_dict!


  0%|          | 0/3 [00:00<?, ?it/s]

/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/sc

,dist\ntest_df,dist\nvalid_df,dist meaning pval\ntest_df,dist meaning pval\nvalid_df,dist meaning stat\ntest_df,dist meaning stat\nvalid_df,marginal gauss\ntest_df,marginal gauss\nvalid_df,normality pval test_df,normality pval valid_df,normality stat test_df,normality stat valid_df
t+1,0.17143 ± 0.022,0.1868 ± 0.025,0.0 ± 0.000,0.0 ± 0.000,-0.05725 ± 0.003,-0.07431 ± 0.005,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,4.40893 ± 0.604,1.54139 ± 0.056
t+2,0.12924 ± 0.016,0.13988 ± 0.017,0.0 ± 0.000,0.0 ± 0.000,-0.04643 ± 0.002,-0.06284 ± 0.002,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,4.04644 ± 0.679,1.51848 ± 0.111
t+3,0.14986 ± 0.008,0.16193 ± 0.008,0.0 ± 0.000,0.0 ± 0.000,-0.03978 ± 0.001,-0.06102 ± 0.003,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,3.89029 ± 0.704,1.46618 ± 0.103
